# Introducción
## Objetivo
Este programa se tuiliza para asignar grupos a estudiantes siguiendo las prioridades:
* Todos los estudiantes (o la mayor cantidad posible), deberían ser asignados a un grupo que hayan marcado como deseable
* Los estudiantes que presentan certificado laboral/deportivo recibem prioridad al momento de la asignación grupos

## Resultado
El programa genera como archivo de salida una tabla en la que se indica el grupo asignado a cada estudiante. En una columna adicional se indica si el turno asignado se encuentra entre los elegidos por el estudiante.

## Reglas
### Sobre los grupos
* Existe un numero $n$ de grupos
* Cada grupo $n_i$ tiene un numero determinado de cupos $c_i$
* Los grupos estan agregados en $S$ *supergrupos* que pueden corresponder a turnos (Matutino, Vespertino, Nocturno), salones de clase, centros de estudio, etc. Cada grupo $n_i$ pertenece a un único supergrupo $S_j$

### Sobre los estudiantes
* Existen $m$ estudiantes. Identificados por un número de cedula o pasaporte.
* Cada estudiante debe elegir al menos 1 grupo en al menos 2 supergrupos diferentes (Todos los estudiantes presentan como mínimo 2 opciones de grupo y 2 opciones de supergrupo).
* Los estudiantes pueden presentar certificado laboral o deportivo. En este caso los estudiantes tienen prioridad en la selección de grupos

# Parametros
A continuación se detallan los parametros necesarios para ejecutar el programa. Algunos de estos pueden tener un valor por defecto.

In [7]:
# Parametros del programa
# Parametros de los grupos
archivo_grupos = 'Grupos_Disponibles.txt'
columna_supergrupos = 'Turno'
columna_min_estudiantes = 'Min Estudiantes'
columna_max_estudiantes = 'Max Estudiantes'

# Parametros de los estudiantes
archivo_estudiantes = 'Formulario_Estudiantes.txt'
columnas_supergrupos = 'Grupos mañana___Grupos tarde___Grupos noche'
columnas_certificados = 'certificado'
columnas_identificacion = 'Cedula___pasaporte'

In [8]:
# Modulos
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [ ]:
# load data
def preprocess_groups(file, sg_col, min_students_col, max_students_col):
    groups = pd.read_csv(file, sep='\t', index_col=0)
    groups['SG'] = groups[sg_col].replace({val:idx for idx, val in enumerate(groups.Turno.unique())})
    
    # ensure min and max student columns have standard names
    groups['min_students'] = groups[min_students_col]
    groups['max_students'] = groups[max_students_col]
    return groups

def preprocess_students(file, id_cols):
    students = pd.read_csv(file, sep='\t')
    
    id_cols = id_cols.split('___')
    
    # identify ID col for each students    
    students['ID'] = ''
    students['Doc_type'] = ''

    for col in id_cols[::-1]:
        col_ids = students.loc[~students[col].isna(), col]
        students.loc[col_ids.index, 'ID'] = col_ids.values
        students.loc[col_ids.index, 'Doc_type'] = col
    # format ids, remove dots and dashes
    students['ID'] = students.ID.apply(lambda x : re.sub('[.]', '', re.sub('-.*', '', x))).apply(lambda x : x[:7])
    return students

# get group demand
def build_demand_table(students, groups, sg_cols):
    sg_cols = sg_cols.split('___')
    uniq_students = students.ID.unique()
    demand_tab = pd.DataFrame(False, index=uniq_students, columns=groups.index)
    sg_demand_tab = pd.DataFrame(0, index=uniq_students, columns=groups.SG.unique())
    
    # mark groups demanded by each student
    for ID, subtab in students.groupby('ID'):
        # merge all group options
        student_opts = ','.join(subtab[sg_cols].fillna('').agg(','.join, axis=1).values)
        # remove weird values, leading commas, empty spaces
        student_opts = re.sub('^,', '', re.sub(',$', '', re.sub(',,+', ',', re.sub(' ','', student_opts))))
        # generate options array
        try:
            student_opts = np.unique(student_opts.split(',')).astype(int)
            # mark student options
            demand_tab.loc[ID, student_opts] = True
        except:
            pass

    # remove weird groups (error when registering option number)
    weird_groups = demand_tab.drop(columns=groups.index)
    demand_tab = demand_tab[groups.index]

    # fill sg demand tab
    for sg, subtab in groups.groupby('SG'):
        sg_demand_tab[sg] = demand_tab[subtab.index].sum(axis=1)
    return demand_tab, sg_demand_tab, weird_groups

# build priority series
def get_certified_students(students, cert_cols):
    uniq_students = students.ID.unique()
    # establish priority students (value 0 so they appear first when sorting by priority)
    certified = pd.Series(False, index=uniq_students)
    certified[students.loc[~students[cert_cols].isna().values, 'ID'].unique()] = True
    return certified

def get_infringing_students(sg_demand, certified, min_groups, min_sg):
    infractions = (sg_demand > min_groups).sum(axis=1) < min_sg
    infractions = infractions & ~certified
    infractions = infractions.loc[infractions].index
    return infractions

def get_no_group_students(demand):
    no_group_students = (demand.sum(axis=1) == 0)
    no_group_students.loc[no_group_students].index
    return no_group_students

def build_priority(students, cert_cols, demand, sg_demand, min_groups=2, min_sg=2):
    certified = get_certified_students(students, cert_cols)
    priority = certified.apply(lambda x : 0 if x else 1)
    
    priority.loc[get_infringing_students(sg_demand, certified, min_groups, min_sg)] = 2
    priority.loc[get_no_group_students(demand)] = 3
    return priority

# assign students
def double_sorting(demand_tab, groups, priority):
    # calculate max iterations (size of largest group)
    max_iterations = groups.loc[demand_tab.columns].max_students.max()
    # prepare placement table
    placement_tab = pd.DataFrame(False, index=demand_tab.index, columns=demand_tab.columns)

    # count student options and detect certified
    student_data = demand_tab.sum(axis=1).to_frame(name='options')
    student_data['priority'] = priority
    # sort by certification status & options
    student_data.sort_values(['priority', 'options'])

    def sort_groups(dtab):
        # sort demand table by group demand, filter out depleted groups
        group_demand = dtab.sum(axis=0).sort_values()
        group_demand = group_demand[group_demand > 0]
        dtab_out = dtab[group_demand.index].copy()
        return dtab_out
    demand_tab2 = sort_groups(demand_tab)

    # iterate
    for i in range(max_iterations):
        # always start placing by the least required groups
        for grp in demand_tab2.columns:
            # check that group hasn't been depleted in the current iteration
            if demand_tab2[grp].sum() > 0:
                grp_student = demand_tab2.index[np.argmax(demand_tab2[grp])] # select the most prioritary student for the current group (certified & least options)
                # place selected student and remove them from the demand table
                placement_tab.loc[grp_student, grp] = True
                demand_tab2.drop(index=grp_student, inplace=True)
        # recalculate demand, resort groups by least demand, drop not demanded groups
        demand_tab2 = sort_groups(demand_tab2)

    # announce total unplaced students
    total_students = demand_tab.shape[0]
    placed_students = placement_tab[placement_tab.any(axis=1)].index.values
    unplaced_students = demand_tab2
    return placement_tab, unplaced_students

# accomodate unplaced students
def transfer(placement_tab, to_transfer, max_students=35):
    # locate available places for students to transfer
    avail_places = max_students - placement_tab[to_transfer.columns].sum(axis=0)
    avail_places = avail_places[avail_places > 0].sort_values(ascending=False)

    # sort students by options (ascending)
    to_transfer = to_transfer.iloc[np.argsort(to_transfer.sum(axis=1))].copy()
    # arrange transfers
    transfer_series = pd.Series(-1, index=to_transfer.index)
    for grp, places in avail_places.items():
        group_students = to_transfer[grp]
        group_students = group_students.loc[group_students].iloc[:places].index.values
        transfer_series[group_students] = grp
        to_transfer.drop(index=group_students, inplace=True)
    transfer_series = transfer_series[transfer_series > -1]
    
    # move students
    placement_tab2 = placement_tab.copy()
    placement_tab2.loc[transfer_series.index] = False
    for student, grp in transfer_series.items():
        placement_tab2.loc[student, grp] = True
    return placement_tab2

def get_moveable_students(placement_tab, demand_tab, max_students=35):
    student_counts = placement_tab.sum(axis=0)
    full_groups = student_counts[student_counts == max_students].index.values
    students_in_full = placement_tab[placement_tab[full_groups].any(axis=1)].index.values

    moveable_students = demand_tab.loc[students_in_full].drop(columns=full_groups)
    moveable_students = moveable_students.loc[moveable_students.any(axis=1), moveable_students.any(axis=0)]
    return moveable_students

def accomodate(placement_tab, unplaced_students, demand_tab, max_students=35):
    # move unplaced students into the freed places
    moveable = get_moveable_students(placement_tab, demand_tab, max_students)

    # transfer moveable students
    placement_tab2 = transfer(placement_tab, moveable, max_students)

    # place unplaced students
    placement_tab2 = transfer(placement_tab2, unplaced_students, max_students)
    unplaced2 = placement_tab2.loc[placement_tab.any(axis=1)]
    return placement_tab2, unplaced2

# construct output
def build_out_tables(placement_tab, demand_tab):
    assigned_places = pd.Series((placement_tab.to_numpy().astype(int) * placement_tab.columns.to_numpy()).sum(axis=1),
                                index=placement_tab.index, name='Grupo asignado')
    students_per_group = placement_tab.sum(axis=0).to_frame(name='Estudiantes')
    unplaced_students = demand_tab.loc[placement_tab.loc[~placement_tab.any(axis=1)].index]
    unplaced_students = unplaced_students.loc[:, unplaced_students.any(axis=0)]
    unplaced_students = pd.DataFrame(unplaced_students.to_numpy().astype(int),
                                     index=unplaced_students.index,
                                     columns=unplaced_students.columns)
    return assigned_places, students_per_group, unplaced_students


In [10]:
def main(students_file,
         groups_file,
         out_prefix,
         id_cols,
         sg_students,
         sg_groups,
         min_students_col,
         max_students_col,
         cert_cols,
         min_groups=2,
         min_sg=2):
    # load data
    students = preprocess_students(students_file, id_cols)
    groups = preprocess_groups(groups_file, sg_groups, min_students_col, max_students_col)

    # build demand
    demand, sg_demand, weird_demand = build_demand_table(students, groups, sg_students)

    # build priority
    priority = build_priority(students, cert_cols, demand, sg_demand, min_groups, min_sg)

    # distribute
    placement, unplaced = double_sorting(demand, groups, priority)

    # acomodate
    placement2, unplaced2 = accomodate(placement, unplaced, demand, max_students=groups.max_students.max())

    # construct output
    assigned_places, students_per_group, unplaced_students = build_out_tables(placement2, demand)
    assigned_places.to_csv(f'{out_prefix}_grupos_asignados.csv')
    students_per_group.to_csv(f'{out_prefix}_estudiantes_por_grupo.csv')
    unplaced_students.to_csv(f'{out_prefix}_estudiantes_no_asignados.csv')
    return
    